In [1]:
import astroprov

from astropy.io import fits
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table
from astropy.table import Column
from astropy.io import fits
from astropy.io.ascii import SExtractor
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy import wcs

from astroquery.simbad import Simbad

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from photutils import DAOStarFinder
from photutils import CircularAperture
import os
import subprocess

import image_registration
from image_registration import chi2_shift
from image_registration.fft_tools import shift
import pandas
import collections

from reproject import reproject_interp
from astropy.utils.data import get_pkg_data_filename
from astropy.wcs import WCS

In [18]:
imageDirectory = '/data/mj1e16/kepler/cal/'
medianDir = '/data/mj1e16/kepler/medians/'
dirlist = os.listdir(imageDirectory)
dirlist = [x for x in dirlist if x[0] != '.']
imagelist = [imageDirectory+x for x in dirlist]


In [3]:
def getImageData(imageName,extension):
    hdu = fits.open(imageName)
    imageData = hdu[extension].data
    return imageData

In [4]:
def correctExtensionOrder(imageName):
    hdu_list = fits.open(imageName)
    extensionOrder = np.zeros(len(hdu_list))
    for channel in range(1,len(hdu_list)):
        hdr = hdu_list[channel].header
        location = hdr['SKYGROUP'] # independant location
        extensionOrder[location] = channel # ordered from 0-84 channel with extension in place of name
    return extensionOrder

In [5]:
def medianAlign(im1_aligned_to_im2,hdulist):
    shaperoni = im1_aligned_to_im2.shape
    xsegments = int(shaperoni[0]/100)
    ylen = shaperoni[0] 

    alignedHDU = [hdulist[0]]
    im2 = hdulist[0]
    for image in range(1,len(hdulist)):
        im1 = hdulist[image]
        yoff,xoff = image_registration.cross_correlation_shifts(im1,im2)
        im1_aligned_to_im2 = np.roll(np.roll(im1,int(yoff),1),int(xoff),0)
        alignedHDU.append(im1_aligned_to_im2)

    output = np.zeros(shaperoni)
    for i in range(100):
        imageSegment = []
        j = i * xsegments
        k = j + xsegments
        for imData in range(len(alignedHDU)):
            imageSegment.append(alignedHDU[imData][j:k,:])

        output[j:k,:] = np.median(imageSegment,axis=0)
    if k != shaperoni[1]: # is this the correct way round?
        imageSegment = []
        print(k,shaperoni[1],'k - shaperoni')
        for imData in range(len(alignedHDU)):
            imageSegment.append(alignedHDU[imData][k:shaperoni[0],:])
        output[k:shaperoni[0],:] = np.median(imageSegment,axis=0)

    
    return output


In [6]:
def findTemplateImage(imagelist,channelList,channel,ximage=1070.,yimage=1140.):
    image = 0
    averageFlux = 0

    for refimage in range(len(imagelist)):
        chanNoList = int(channelList[refimage][channel])
        chanNoImName = imagelist[refimage]
        im2 = getImageData(chanNoImName,chanNoList)        
        averageFlux = int((sum(sum(im2)))/(ximage*yimage))
        if averageFlux > 1:
            break
        
    goodImage = {'chanNo':chanNoList,'chanNoImName':chanNoImName}
    return goodImage

In [7]:
def findSpecificTemplateImage(imagelist,channelList,channel,specificChannel,ximage=1070.,yimage=1140.):
    image = 0
    averageFlux = 0

    for refimage in range(len(imagelist)):
        chanNoList = int(channelList[refimage][channel])
        if chanNoList == specificChannel:
            chanNoImName = imagelist[refimage]
            im2 = getImageData(chanNoImName,chanNoList)
            averageFlux = int((sum(sum(im2)))/(ximage*yimage))
        if averageFlux > 1:
            break
        
    goodImage = {'chanNo':chanNoList,'chanNoImName':chanNoImName}
    return goodImage

In [8]:
def newMedian(imageList,channelList,outfile='blah',ximage=1070,yimage=1140):
    
    image_concat = []
    medianlist = []

    ccds = [44,63,79]
    
    for channel in range(1,len(channelList[0])):
    #for channel in ccds:
        
        hdulist = []
        goodImage = findTemplateImage(imagelist,channelList,channel)
        chanNoList = [goodImage['chanNo']]
        chanNoImName = [goodImage['chanNoImName']]
        im2 = getImageData(chanNoImName[0],chanNoList[0])
        
        for image in range(1,len(imagelist)):
            chanNo = int(channelList[image][channel])
            im1 = getImageData(imagelist[image],chanNo)      
            averageFlux = int((sum(sum(im1)))/(ximage*yimage))
            if averageFlux > 50:
                
                hdulist.append(im1)
                
            
        output = medianAlign(im1,hdulist)
        print(chanNoList)
        print(chanNoImName)
        hdu = fits.PrimaryHDU(output)
        hdu.writeto(outfile+'_{}.fits'.format(channel), clobber=True)
        medianlist.append(outfile+'_{}.fits'.format(channel))
        
        
    return medianlist
    
    

In [9]:
def selectiveMedian(imageList,channelList,outfile='blah',ximage=1070,yimage=1140):
    
    image_concat = []
    medianlist = []

    #ccds = [44,63,79]
    for channel in range(1,len(channelList[0])):
    #for channel in ccds:
        hdulist = [[],[],[],[]]
        offSet =  1000
        refimage = 0
        image = 0
        
        goodImage = findTemplateImage(imagelist,channelList,channel)
        chanNoList = [goodImage['chanNo']]
        chanNoImName = [goodImage['chanNoImName']]
        hdulist[0].append(getImageData(chanNoImName[0],chanNoList[0]))
        for image in range(1,len(imagelist)):
            chanNo = int(channelList[image][channel])
            if chanNo in chanNoList: 
                loc = chanNoList.index(chanNo)
                im1 = getImageData(imagelist[image],chanNo)
                
                averageFlux = int((sum(sum(im1)))/(ximage*yimage))
                if averageFlux > 50:
                    if chanNo == chanNoList[loc]:
                        hdulist[loc].append(im1)

            else:
                goodImage = findSpecificTemplateImage(imagelist,channelList,channel,chanNo)
                chanNoList.append(goodImage['chanNo'])
                chanNoImName.append(goodImage['chanNoImName'])
                hdulist[len(chanNoList)-1].append(getImageData(chanNoImName[-1],chanNoList[-1]))
                

        for loc in range(len(hdulist)):
            output = medianAlign(im1,hdulist[loc])        
            print(chanNoList)
            print(chanNoImName)
            hdu = fits.PrimaryHDU(output)
            hdu.writeto(outfile+'_{}_{}.fits'.format(channel,chanNoList[loc]), clobber=True)
            medianlist.append(outfile+'_{}_{}.fits'.format(channel,chanNoList[loc]))
        
        
    return medianlist
    
    

In [10]:
extensionOrders = []
for image in imagelist:
    extensionOrders.append(correctExtensionOrder(image))

In [11]:
selectiveMedian(imagelist,extensionOrders,outfile='/data/mj1e16/kepler/medians/meidian')

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/image_registration/fft_tools/convolve_nd.py:269: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  bigarray[arrayslices] = array
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/image_registration/fft_tools/convolve_nd.py:270: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  bigkernel[kernslices] = kernel
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/image_registration/fft_tools/convolve_nd.py:325: FutureWarning: Using a non-tuple sequenc

(1000, 1132, 'k - shaperoni')
[53, 81, 29, 1]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']


(1000, 1132, 'k - shaperoni')
[53, 81, 29, 1]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[53, 81, 29, 1]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[53, 81, 29, 1]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[54, 82, 30, 2]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', 

(1000, 1132, 'k - shaperoni')
[13, 73, 69, 9]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[13, 73, 69, 9]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[13, 73, 69, 9]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[13, 73, 69, 9]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', 

(1000, 1132, 'k - shaperoni')
[76, 72, 12, 16]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[76, 72, 12, 16]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[57, 65, 25, 17]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[57, 65, 25, 17]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fit

...

(1000, 1132, 'k - shaperoni')
[45, 21, 37, 61]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[45, 21, 37, 61]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[46, 22, 38, 62]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[46, 22, 38, 62]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fit

(1000, 1132, 'k - shaperoni')
[9, 13, 73, 69]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[9, 13, 73, 69]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[9, 13, 73, 69]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[9, 13, 73, 69]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', 

(1000, 1132, 'k - shaperoni')
[72, 12, 16, 76]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[72, 12, 16, 76]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[49, 5, 33, 77]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009170043915_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[49, 5, 33, 77]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009170043915_ffi-cal.fits'

(1000, 1132, 'k - shaperoni')
[32, 4, 56, 84]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[32, 4, 56, 84]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[32, 4, 56, 84]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[32, 4, 56, 84]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits', '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits', 

['/data/mj1e16/kepler/medians/meidian_1_53.fits',
 '/data/mj1e16/kepler/medians/meidian_1_81.fits',
 '/data/mj1e16/kepler/medians/meidian_1_29.fits',
 '/data/mj1e16/kepler/medians/meidian_1_1.fits',
 '/data/mj1e16/kepler/medians/meidian_2_54.fits',
 '/data/mj1e16/kepler/medians/meidian_2_82.fits',
 '/data/mj1e16/kepler/medians/meidian_2_30.fits',
 '/data/mj1e16/kepler/medians/meidian_2_2.fits',
 '/data/mj1e16/kepler/medians/meidian_3_55.fits',
 '/data/mj1e16/kepler/medians/meidian_3_83.fits',
 '/data/mj1e16/kepler/medians/meidian_3_31.fits',
 '/data/mj1e16/kepler/medians/meidian_3_3.fits',
 '/data/mj1e16/kepler/medians/meidian_4_56.fits',
 '/data/mj1e16/kepler/medians/meidian_4_84.fits',
 '/data/mj1e16/kepler/medians/meidian_4_32.fits',
 '/data/mj1e16/kepler/medians/meidian_4_4.fits',
 '/data/mj1e16/kepler/medians/meidian_5_33.fits',
 '/data/mj1e16/kepler/medians/meidian_5_77.fits',
 '/data/mj1e16/kepler/medians/meidian_5_49.fits',
 '/data/mj1e16/kepler/medians/meidian_5_5.fits',
 '/da

In [12]:
newMedian(imagelist,extensionOrders,outfile='/data/mj1e16/kepler/medians/meidian')

(1000, 1132, 'k - shaperoni')
[53]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[54]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[55]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[56]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[33]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[34]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[35]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[36]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[13]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[14]
['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits']
(1000, 1132, 'k - shaperoni')
[15]
['/data/mj1e16/kepler/cal

['/data/mj1e16/kepler/medians/meidian_1.fits',
 '/data/mj1e16/kepler/medians/meidian_2.fits',
 '/data/mj1e16/kepler/medians/meidian_3.fits',
 '/data/mj1e16/kepler/medians/meidian_4.fits',
 '/data/mj1e16/kepler/medians/meidian_5.fits',
 '/data/mj1e16/kepler/medians/meidian_6.fits',
 '/data/mj1e16/kepler/medians/meidian_7.fits',
 '/data/mj1e16/kepler/medians/meidian_8.fits',
 '/data/mj1e16/kepler/medians/meidian_9.fits',
 '/data/mj1e16/kepler/medians/meidian_10.fits',
 '/data/mj1e16/kepler/medians/meidian_11.fits',
 '/data/mj1e16/kepler/medians/meidian_12.fits',
 '/data/mj1e16/kepler/medians/meidian_13.fits',
 '/data/mj1e16/kepler/medians/meidian_14.fits',
 '/data/mj1e16/kepler/medians/meidian_15.fits',
 '/data/mj1e16/kepler/medians/meidian_16.fits',
 '/data/mj1e16/kepler/medians/meidian_17.fits',
 '/data/mj1e16/kepler/medians/meidian_18.fits',
 '/data/mj1e16/kepler/medians/meidian_19.fits',
 '/data/mj1e16/kepler/medians/meidian_20.fits',
 '/data/mj1e16/kepler/medians/meidian_21.fits',
 

In [13]:
def hotpants(inim,tmplim,outim):
    subprocess.call(['./hotpants','-inim',inim,'-tmplim',tmplim,'-outim',outim,'-v','0'])
    

In [17]:
medianDir

'/data/mj1e16/kepler/newDiff/'

In [14]:
imagelist = os.listdir('/data/mj1e16/kepler/cal/')
imagelist = [x for x in imagelist if x[-4:]=='fits']
print(imagelist)

['kplr2012121122500_ffi-cal.fits', 'kplr2011240181752_ffi-cal.fits', 'kplr2009322233047_ffi-cal.fits', 'kplr2010111125026_ffi-cal.fits', 'kplr2011177110110_ffi-cal.fits', 'kplr2013011160902_ffi-cal.fits', 'kplr2011271191331_ffi-cal.fits', 'kplr2010174164113_ffi-cal.fits', 'kplr2012088132324_ffi-cal.fits', 'kplr2010265195356_ffi-cal.fits', 'kplr2009170043915_ffi-cal.fits', 'kplr2009114174833_ffi-cal.fits', 'kplr2011303191211_ffi-cal.fits', 'kplr2011334181008_ffi-cal.fits', 'kplr2009260000800_ffi-cal.fits', 'kplr2009114204835_ffi-cal.fits', 'kplr2012341215621_ffi-cal.fits', 'kplr2009115080620_ffi-cal.fits', 'kplr2010296192119_ffi-cal.fits', 'kplr2013038133130_ffi-cal.fits', 'kplr2011116104002_ffi-cal.fits', 'kplr2010140101631_ffi-cal.fits', 'kplr2010019225502_ffi-cal.fits', 'kplr2012310200152_ffi-cal.fits', 'kplr2013098115308_ffi-cal.fits', 'kplr2010020005046_ffi-cal.fits', 'kplr2010234192745_ffi-cal.fits', 'kplr2012179140901_ffi-cal.fits', 'kplr2009351005245_ffi-cal.fits', 'kplr20112081

In [21]:
os.chdir('/data/mj1e16/kepler/hotpants-master/')
for imNo in range(len(imagelist)):
    for channel in range(1,85):
        ccdChannel = int(extensionOrders[imNo][channel])
        inim = imagelist[imNo]+'[{}]'.format(ccdChannel)
        tmplim = medianDir+'meidian_{}_{}.fits'.format(channel,ccdChannel)
        outim = '/data/mj1e16/kepler/properDiff2/diff_imNo{}_channel{}_ccd{}.fits'.format(imNo,channel,ccdChannel)
        subprocess.call(['./hotpants','-inim',inim,'-tmplim',tmplim,'-outim',outim,'-v','0'])

In [19]:
# os.chdir('/data/mj1e16/kepler/hotpants-master/')
# for imNo in range(len(imagelist)):
#     for channel in range(1,85):
#         for med in range(4):
#             ccdChannel = int(extensionOrders[imNo][channel])
#             inim = '/data/mj1e16/kepler/cal/'+imagelist[imNo]+'[{}]'.format(ccdChannel)
#             tmplim = medianDir+'meidian_{}_{}.fits'.format(channel,ccdChannel)
#             outim = '/data/mj1e16/kepler/properDiff2/diff_imNo{}_channel{}.fits'.format(imNo,channel)
#             subprocess.call(['./hotpants','-inim',inim,'-tmplim',tmplim,'-outim',outim,'-v','0'])

KeyboardInterrupt: 